### Startup imports

In [3]:
import pandas as pd

from autoqchem.molecule import molecule
from autoqchem.db_functions import *
from autoqchem.openbabel_functions import *

import hashlib
import logging
logging.basicConfig(level=logging.INFO)

RDKit WARNING: [11:26:39] Enabling RDKit 2019.09.3 jupyter extensions


In [4]:
tags_coll = db_connect("tags")
mols_coll = db_connect("molecules")
log_files_coll = db_connect("log_files")

In [5]:
# INPUT parameters
tag = "PARC"

In [6]:
# fetch molecules with that tag
mol_df = db_select_molecules(tags=[tag])

C:/Users/AndrzejZuranski/Dropbox/DataX_PU/github/auto-qchem\autoqchem\db_functions.py:154: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = pd.concat([grouped['metadata', 'molecule_id', 'name'].first(),


In [49]:
# helper function to create file names the same way ACQ does
def mol_fs_name(can):
    mol = input_to_OBMol(can, "string", "smi")
    return f"{mol.GetFormula()}_{hashlib.md5(can.encode()).hexdigest()[:4]}"

In [50]:
# add fs_name to the table
mol_df['file_base_name'] = mol_df['can'].map(mol_fs_name)

In [51]:
# get a cursor that iterates over the log files
cursor = log_files_coll.find({'molecule_id' :{"$in": mol_df.molecule_id.tolist()}}, {'log':1, 'can':1})

In [52]:
# path for file storage
path = 'C:/Users/AndrzejZuranski/Downloads/for_Lucy'


# dictionary to keep trach of conformer numbers
m_dict = {m:0 for m in mol_df.can.tolist()}

for l in cursor:
    can, log = l['can'], l['log']
    fs_name = mol_fs_name(can)
    
    print (can, m_dict[can])
    
    with open(f"{path}/{fs_name}_{m_dict[can]}.log", "w") as f:
        f.write(log)
    
    m_dict[can] +=1 

C1CCC(CC1)/N=C/C=N/N1CCCCC1 0
C1CCC(CC1)/N=C/C=N/N1CCCCC1 1
C1CCC(CC1)/N=C/C=N/N1CCCCC1 2
C1CN=C(O1)C1=NCCO1 0
C1CN=C(O1)C1=NCCO1 1
C1CN=C(O1)c1ccc2c(n1)cccc2 0
C1CN=C(O1)c1cccc(n1)C1=NCCO1 0
C1CN=C(O1)c1cccc(n1)C1=NCCO1 1
CC(c1cc(nc(c1)c1nccc(c1)C(C)(C)C)c1nccc(c1)C(C)(C)C)(C)C 0
CC(c1cc(nc(c1)c1nccc(c1)C(C)(C)C)c1nccc(c1)C(C)(C)C)(C)C 1
CC(c1cc(nc(c1)c1nccc(c1)C(C)(C)C)c1nccc(c1)C(C)(C)C)(C)C 2
CC(c1ccnc(c1)c1nccc(c1)C(C)(C)C)(C)C 0
CC(c1ccnc(c1)c1nccc(c1)C(C)(C)C)(C)C 1
CC(c1ccnc(c1)c1nccc(c1)C(C)(C)C)(C)C 2
CN(CCN(C)C)C 0
CN(CCN(C)C)C 1
CN(CCN(C)C)C 2
CN(c1cccc2c1c(ccc2)N(C)C)C 0
CNC(=N)c1ccccn1 0
CNC(=N)c1ccccn1 1
COc1ccc(nc1)C(=N)N 0
COc1ccc(nc1)C(=N)N 1
COc1ccc(nc1)C(=N)N 2
COc1ccnc(c1)C(=N)N 0
COc1ccnc(c1)C(=N)N 1
COc1ccnc(c1)C(=N)N 2
COc1ccnc(c1)c1nccc(c1)OC 0
COc1ccnc(c1)c1nccc(c1)OC 1
COc1ccnc(c1)c1nccc(c1)OC 2
Cc1ccc(cc1)S(=O)(=O)N[C@H]1CCCC[C@@H]1NS(=O)(=O)c1ccc(cc1)C 0
Cc1ccc(cc1)S(=O)(=O)N[C@H]1CCCC[C@@H]1NS(=O)(=O)c1ccc(cc1)C 1
Cc1ccc(cc1)S(=O)(=O)N[C@H]1CCCC[C@@H]1NS(=

In [53]:
mol_df.drop(['molecule_id', '_ids'], axis=1).to_csv(f"{path}/legend.csv")